This approach is similar to the one I have used in Lab 03. I am using the text itself as the input and I am getting accuracy of just over 30%.

Notably, when I use just the prefix or suffix (the part before or after the separator; or for memers known as top text and bottom text), the accuracy is still over 20%. I have considered clasifing them separately and then combining the results into one prediction for the whole meme, however I prioritized other experiments. It is therefore left as an exercise for the reader.

Evaluation based on parts of speach tagging has accuracy of 6.68% and 11.63% depeneding on which tagset is used, whether universal or NLTK default.


In [6]:
# !pip install transformers==4.2.2 tqdm numpy scikit-learn nltk pandas

In [7]:
# !nvidia-smi

In [8]:
# from google.colab import files

# uploaded = files.upload()
# uploaded

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [10]:
dataset = pd.read_csv('all.csv')
dataset = dataset[["MemeName", "Text", "prefix", "suffix", "TextWithoutSeparator_tagged_universal", "TextWithoutSeparator_tagged_None"]]
dataset.head(5)

,MemeName,Text,prefix,suffix,TextWithoutSeparator_tagged_universal,TextWithoutSeparator_tagged_None
0,Y U No,Forever alone guy <sep> y u no get cat,Forever alone guy,y u no get cat,ADV ADV ADJ NOUN VERB DET NOUN NOUN,RB RB JJ NNS VBP DT NN NN
1,Y U No,TEAMMATES <sep> Y U NO REVIVE ME?,TEAMMATES,Y U NO REVIVE ME?,NOUN NOUN NOUN NOUN NOUN NOUN .,NNP NNP NNP NNP NNP NNP .
2,Y U No,GIRLS <sep> Y U SO COMPLICATED ??!,GIRLS,Y U SO COMPLICATED ??!,NOUN NOUN NOUN NOUN NOUN . . .,NNP NNP NNP NNP NNP . . .
3,Y U No,I 'like' all your pics <sep> Y U No have sex w...,I 'like' all your pics,Y U No have sex with me?,PRON VERB . DET PRON NOUN NOUN NOUN NOUN VERB ...,PRP MD '' DT PRP$ NNS NNP NNP NNP VBP NN IN PRP .
4,Y U No,girls <sep> y u no stop making duck faces?!,girls,y u no stop making duck faces?!,NOUN VERB ADJ DET NOUN VERB ADJ VERB . .,NNS VBP JJ DT NN VBG JJ VBZ . .


In [26]:


def svc_complete(features: pd.Series, labels: pd.Series): 

    vectorizer = TfidfVectorizer()

    # X -> features, y -> label
    X = vectorizer.fit_transform(features)
    y = labels

    # https://www.geeksforgeeks.org/multiclass-classification-using-scikit-learn/
    # dividing X, y into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

    # training a linear SVM classifier
    from sklearn.svm import SVC, LinearSVC
    svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
#     svm_model_linear = LinearSVC().fit(X_train, y_train)
    svm_predictions = svm_model_linear.predict(X_test)

    # model accuracy for X_test 
    accuracy = svm_model_linear.score(X_test, y_test)

    # creating a confusion matrix
    cm = confusion_matrix(y_test, svm_predictions)

    return svm_model_linear, accuracy, cm



In [12]:
m_text, accuracy, cm_text = svc_complete(dataset.Text, dataset.MemeName)
accuracy

0.30458666666666667

In [27]:
# This cell was calculated with LinearSVC instead of SVC
# That massively improved speed, but slightly (0.5%) lowered accuracy.
# _, accuracy, _ = svc_complete(dataset.Text, dataset.MemeName)
# accuracy

0.2966933333333333

In [13]:
m_prefix, accuracy, cm_prefix = svc_complete(dataset.prefix, dataset.MemeName)
accuracy

0.20181333333333334

In [14]:
m_suffix, accuracy, cm_suffix = svc_complete(dataset.suffix, dataset.MemeName)
accuracy

0.20602666666666666

In [15]:
_, accuracy, _ = svc_complete(dataset.TextWithoutSeparator_tagged_universal, dataset.MemeName)
accuracy

0.06682666666666667

In [16]:
_, accuracy, _ = svc_complete(dataset.TextWithoutSeparator_tagged_None, dataset.MemeName)
accuracy

0.11632